<a href="https://colab.research.google.com/github/itonse/google-colab-archive/blob/main/goohaeyou_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [15]:
## 도로명 주소 변환 (서울 데이터)

import pandas as pd
import os

# 데이터 로드
file_path = "/content/juso/seoul.csv"
data = pd.read_csv(
    file_path,
    encoding='cp949',
    sep='|',
    usecols=range(13)
)

# 열 이름 설정
data.columns = ['id', 'code', '시도', 'eng_sido', '시군구', 'eng_sigungu', 'unused1', 'unused2',
                'road_code', '도로명', 'eng_roadname', 'building_main_no', 'building_sub_no']

# 문자열로 변환하고 NaN 값을 빈 문자열로 대체
data = data.fillna('').astype(str)

# 필요한 열 추출 및 주소 포맷팅
def format_address(row):
    building_sub_no = str(int(float(row['building_sub_no']))) if row['building_sub_no'] else ''
    address_parts = [row['시도'], row['시군구'], row['도로명'], building_sub_no]
    formatted_address = " ".join(part for part in address_parts if part)  # 부분이 비어있지 않으면 포함
    return formatted_address

data['FormattedAddress'] = data.apply(format_address, axis=1)

# 중복된 행 제거
data.drop_duplicates(subset='FormattedAddress', keep='first', inplace=True)

# 결과 데이터를 새로운 Excel 파일로 저장
output_path = "/content/result/seoul.xlsx"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
data.to_excel(output_path, index=False, columns=['FormattedAddress'])

<ipython-input-15-2421bba7f4d2>:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(


       id code     시도 eng_sido  시군구 eng_sigungu unused1 unused2  \
0  135965  1.0  서울특별시    Seoul  강남구  Gangnam-gu                   
1  135964  1.0  서울특별시    Seoul  강남구  Gangnam-gu                   
2  135964  1.0  서울특별시    Seoul  강남구  Gangnam-gu                   
3  135965  1.0  서울특별시    Seoul  강남구  Gangnam-gu                   
4  135962  1.0  서울특별시    Seoul  강남구  Gangnam-gu                   

        road_code     도로명     eng_roadname building_main_no building_sub_no  \
0  116804166052.0  개포로17길  Gaepo-ro 17-gil              0.0             9.0   
1  116804166053.0  개포로20길  Gaepo-ro 20-gil              0.0            25.0   
2  116803121022.0     논현로      Nonhyeon-ro              0.0            88.0   
3  116803122001.0     개포로         Gaepo-ro              0.0           251.0   
4  116803121022.0     논현로      Nonhyeon-ro              0.0            42.0   

      FormattedAddress  
0   서울특별시 강남구 개포로17길 9  
1  서울특별시 강남구 개포로20길 25  
2     서울특별시 강남구 논현로 88  
3    서울특별시 강남구 개포로 251

In [ ]:
## 지역별 데이터를 무작위로 섞음

import pandas as pd
import numpy as np

# 파일 경로 설정
excel_path = 'result/location.xlsx'

# 엑셀 파일 로드
df = pd.read_excel(excel_path)

# 데이터프레임의 행을 무작위로 섞음
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 섞은 데이터 저장
output_path = 'result/shuffled_location.xlsx'
shuffled_df.to_excel(output_path, index=False)

In [28]:
## title(공고 제목), body(공고 내용) 쌍의 데이터 100만개 생성

import pandas as pd
import random

def generate_dataset(num_samples):
    job_types = [
    # 일상 도움
    "강아지 산책", "강아지 발톱 정리", "고양이 귀청소", "햄스터 톱밥 청소",
    "화분 분갈이", "커튼봉 설치", "여행계획", "기차 예매", "쇼핑 같이",
    "전화 문의 대신", "찢어진 인형 수리", "그늘막 설치", "모루인형 만들기",
    "시험장까지 픽업", "아이 픽업",
    # 정리 및 청소
    "정리정돈", "집 청소", "화단 청소", "이불 빨래", "베란다 창문 닦기",
    "대형 폐기물 운반", "현수막 수거", "백화점 오픈런",
    # 물류 및 배송
    "배달", "짐 이동", "우편물", "헬스기구 운반", "가게에서 물건 픽업",
    "터미널 수화물 픽업", "굿즈 대리구매", "팝업스토어 대리구매", "과일포장 작업",
    "식품 포장", "준비물 전달",
    # 기술 작업
    "정비", "침대 프레임 조립", "시계 약 교체", "커튼봉 설치", "싱크대 수전 교체",
    "워셔액 보충", "차량 와이퍼 교체", "컴퓨터 설치", "노트북 수리", "증명사진 보정",
    # 매장
    "생일선물 포장", "포장", "전단지 부착", "홍보물부착",
    # 사무 및 교육
    "미술학원 일일 보조 강사", "학원 수강생 출석 체크", "수업 자료 정리",
    # 행사
    "체육대회 정리", "스타벅스 프리퀀시 수령 오픈런", "이벤트 장식 설치",
    # 기타
    "단순조립", "헤어 모델", "고양이 방문 시터", "피아노 운반/조율", "유튜브 콘텐츠 촬영",
    "당근마켓 지역 인증", "벌집 제거"
]
    descriptors = [
        "단순", "[바로가능]", "오늘", "지금", "근처 사시는 분", "하루만", "[30분 소요]",
        "경험자 우대!", "추가 모집", "상시", "내일", "[긴급]", "[협의 가능]", "쉬워요/", "[현장 지급]",
        "1시간 이하/", "[10분컷]"
        ]
    actions = [
        "해주실분", "해주세요", "구합니다", "구해요", "하실 분 구해요", "하실 분 있으신가요?", "도와주실 분 찾습니다",
        "해주실분 찾아요", "도와주실 분 구해요", "하실 분?"
    ]
    details1 = [
        "시간 약속 잘 지키시는 분", "경험 있으신 분", "금액 제시도 가능합니다",
        "친구랑 같이 가능", "일찍 끝나도 약속한대로 지급", "성격이 밝으신 분", "초보자 지원하지 말아주세요",
        "시간 조절 가능", "시간/페이 협의 가능"
    ]
    details2 = [
        "1명만 모집합니다", "오래 하실 수 있는 분 좋아요", "자주 하실 수 있으면 좋습니다", "댓글로 문의 해주세요~",
        "승인되시면 전화로 안내드립니다", "많이 지원해주세요", "자신 있는 분 지원하세요",
        "현장에서 바로 지급합니다", "댓글에 질문 남겨주세요.", "승인 후 채팅 답장 부탁합니다.", "채팅으로 자세한 시간약속 잡을게요",
        "장소 이동 가능하신 분이 좋습니다", "책임감있게 해주실 분 환영해요"
    ]

    selected_job_types = random.choices(job_types, k=num_samples)

    titles = [f"{random.choice(descriptors)} {job_type} {random.choice(actions)}"
              for job_type in selected_job_types]
    bodies = [f"{job_type} {random.choice(details1)} \n{random.choice(details2)}"
              for job_type in selected_job_types]

    # 데이터셋 생성
    data = pd.DataFrame({
        "title": titles,
        "body": bodies,
    })

    return data

full_dataset = generate_dataset(1000000)

sava_path = "/content/result/title_body.xlsx"
full_dataset.to_excel(sava_path, index=False)

In [48]:
## job_post 데이터 100만개 bulk insert문 생성

import pandas as pd
import random
from datetime import datetime, date, timedelta

# 파일 경로
input_excel_title_body_path = 'temp/title_body.xlsx'
input_excel_location_path = 'temp/shuffled_location.xlsx'
output_path = 'result/bulk_insert_job_post.sql'

# 파일 로드
title_body_df = pd.read_excel(input_excel_title_body_path)
locations_df = pd.read_excel(input_excel_location_path)

# 지역코드 매핑
region_code_map = {
    "서울": 101,
    "경기": 102,
    "부산": 103,
    "경남": 104,
    "인천": 105,
    "경북": 106,
    "대구": 107,
    "충남": 108,
    "전남": 109,
    "전북특별자치도": 110,
    "충북": 111,
    "강원특별자치도": 112,
    "대전": 113,
    "광주": 114,
    "울산": 115,
    "제주특별자치도": 116,
    "세종특별자치시": 117
}

# 날짜 범위 설정
start_date = date(2024, 8, 2)
end_date = date(2024, 11, 29)
date_range = pd.date_range(start_date, end_date).to_pydatetime().tolist()

# 데이터 생성
created_at = datetime(2024, 7, 31, 17, 0)
modified_at = created_at
application_count = 0
comments_count = 0
increment_view_count = 0
interests_count = 0
closed = False
employed = False
member_id = 1   # 우선은 1로 둔다(나중에 업데이트 예정)

# 데이터 개수 지정
num_samples = 1000000

# SQL 파일 오픈
with open(output_path, 'w') as file:
    # Bulk insert 시작 부분
    file.write("INSERT INTO job_post (id, created_at, modified_at, application_count, closed, comments_count, deadline, employed, increment_view_count, interests_count, job_start_date, location, title, member_id, region_code) VALUES\n")


    for i in range(0, num_samples):
        deadline = random.choice(date_range)
        job_start_date = deadline + timedelta(days=1)
        title = title_body_df.iloc[i, 0]
        location = locations_df.iloc[i, 0]

        # 지역코드 추출
        region = location.split()[0]  # 도로명 주소에서 첫 번째 단어는 시,도
        region_code = region_code_map.get(region, 999)

        # 날짜 포맷을 2024-01-01 형식으로 변경
        deadline_str = deadline.strftime('%Y-%m-%d')
        job_start_date_str = job_start_date.strftime('%Y-%m-%d')

        # 데이터 값을 생성
        sql = f"({i + 1}, '{created_at.strftime('%Y-%m-%d %H:%M:%S.%f')}', '{modified_at.strftime('%Y-%m-%d %H:%M:%S.%f')}', {application_count}, {int(closed)}, {comments_count}, '{deadline_str}', {int(employed)}, {increment_view_count}, {interests_count}, '{job_start_date_str}', '{location}', '{title}', {member_id}, {region_code})"

        # 마지막 데이터가 아니면 쉼표와 줄바꿈 추가
        if i < num_samples - 1:
            sql += ",\n"
        else:
            sql += ";\n"

        file.write(sql)

In [ ]:
## job_post_detail 데이터 100만개 bulk insert문 생성

import pandas as pd

# 파일 경로
excel_title_body_path = 'temp/title_body.xlsx'
output_path = 'result/bulk_insert_job_post_detail.sql'

# 파일 로드
title_body_df = pd.read_excel(excel_title_body_path)

# 데이터 개수 지정
num_samples = 1000000

# 데이터 생성
created_at = '2024-07-31 17:00:00.000000'
modified_at = '2024-07-31 17:00:00.000000'
author = 'user1'  # 우선은 user1로 둔다(나중에 업데이트 예정)

# SQL 파일 오픈
with open(output_path, 'w') as file:
    # Bulk insert 시작 부분
    file.write("INSERT INTO job_post_detail (id, created_at, modified_at, author, body, job_post_id) VALUES\n")

    for i in range(0, num_samples):
        body = title_body_df.iloc[i, 1]
        job_post_id = i + 1

        # 데이터 값을 생성
        sql = f"({i + 1}, '{created_at}', '{modified_at}', '{author}', '{body}', {job_post_id})"

        # 마지막 데이터가 아니면 쉼표와 줄바꿈 추가
        if i < num_samples - 1:
            sql += ",\n"
        else:
            sql += ";\n"

        file.write(sql)

In [54]:
## essential 데이터 100만개 bulk insert문 생성

import random

# 파일 경로 설정
output_sql_path = 'result/bulk_insert_essential.sql'

# 데이터 범위 및 옵션 설정
num_samples = 1000000
genders = ['UNDEFINED'] * 60 + ['MALE'] * 20 + ['FEMALE'] * 20  # 무관6, 남2, 여2 비율로 설정
age_distribution = [0] * 40 + [20] * 30 + [25] * 5 + [30] * 10 + [40] * 5  # 해당 비율로 나이 분포 설정

# SQL 파일 생성
with open(output_sql_path, 'w') as file:
    # Bulk insert 시작 부분
    file.write("INSERT INTO essential (id, gender, min_age, job_post_detail_id) VALUES\n")

    for i in range(1, num_samples + 1):
        gender = random.choice(genders)  # 랜덤으로 성별 선택
        min_age = random.choice(age_distribution)  # 랜덤으로 나이 선택
        job_post_detail_id = i  # 1부터 100만까지 순차적 증가

        # 데이터 값을 생성
        sql = f"({i}, '{gender}', {min_age}, {job_post_detail_id})"

        # 마지막 데이터가 아니면 쉼표와 줄바꿈 추가
        if i < num_samples:
            sql += ",\n"
        else:
            sql += ";\n"

        file.write(sql)

In [ ]:
## wage 데이터 100만개 bulk insert문 생성

import random

# SQL 파일에서 데이터 읽기
def read_sql_file(file_path):
    titles = []
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        for line in file:
            if line.strip().endswith('),') or line.strip().endswith(');'):  # 데이터가 있는 행인지 확인
                parts = line.split(',')
                title = parts[12].strip().strip("'")  # 'title' 열의 값을 추출
                titles.append(title)
    return titles

# wage 데이터 생성
def generate_wage(title, index):
    day_keywords = ["오늘", "지금", "하루만", "내일", "체육대회 정리", "헬스기구 운반", "일일", "그늘막 설치"]
    hour_keywords = ["30분", "1시간", "10분컷", "시계 약 교체", "커튼봉 설치", "기차 예매",
                     "강아지 발톱 정리", "고양이 귀청소", "햄스터 톱밥 청소", "시험장까지 픽업"]

    # pay_basis 설정
    if any(keyword in title for keyword in day_keywords):
        pay_basis = 'TOTAL_DAYS'
        work_days = 1
        work_time = 0
    elif any(keyword in title for keyword in hour_keywords):
        pay_basis = 'TOTAL_HOURS'
        work_days = 1
        work_time = 1
    else:
        pay_basis = random.choice(['TOTAL_HOURS'] * 70 + ['TOTAL_DAYS'] * 30)
        if pay_basis == 'TOTAL_HOURS':
            work_days = 1
            work_time = random.choice([1, 2] * 70 + [3, 4, 5] * 20 + [6, 7, 8] * 10)  # 1,2는 70%, 3~5는 20%, 6~8는 10% 비율로 설정
        else:
            work_days = random.choice([1, 2, 3] * 70 + [4, 5] * 25 + [6, 7] * 5)  # 1~3은 70%, 4,5는 25%, 6,7은 5% 비율로 설정
            work_time = 0

    # cost 설정
    if pay_basis == 'TOTAL_HOURS':
        if work_time in [1, 2]:
            cost = random.choice(range(10000, 35000, 5000))  # 10000에서 30000까지, 5000 단위
        elif work_time in [3, 4, 5]:
            cost = random.choice(range(20000, 45000, 5000))  # 20000에서 40000까지, 5000 단위
        else:
            cost = random.choice(range(40000, 65000, 5000))  # 40000에서 60000까지, 5000 단위
    else:
        if work_days in [1, 2, 3]:
            cost = random.choice(range(20000, 45000, 5000))  # 20000에서 40000까지, 5000 단위
        elif work_days in [4, 5]:
            cost = random.choice(range(30000, 65000, 5000))  # 30000에서 60000까지, 5000 단위
        else:
            cost = random.choice(range(40000, 95000, 5000))  # 40000에서 90000까지, 5000 단위

    wage_payment_method = random.choice(['INDIVIDUAL_PAYMENT', 'SERVICE_PAYMENT'])
    paid = 0  # 모두 false니까 0으로 설정

    return f"({index + 1}, {cost}, {paid}, '{pay_basis}', '{wage_payment_method}', {work_days}, {work_time}, {index + 1}),\n"

# SQL 파일 쓰기
def write_to_sql_file(titles, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write("INSERT INTO wage (id, cost, paid, pay_basis, wage_payment_method, work_days, work_time, job_post_detail) VALUES\n")
        for index, title in enumerate(titles):
            sql_statement = generate_wage(title, index)
            if index < len(titles) - 1:
                file.write(sql_statement)
            else:
                # 마지막 행 처리
                file.write(sql_statement[:-2] + ";\n")

# 파일 경로 설정
input_file_path = 'result/bulk_insert_job_post.sql'
output_file_path = 'result/bulk_insert_wage.sql'

# 파일 읽고 쓰기
job_post_titles = read_sql_file(input_file_path)
write_to_sql_file(job_post_titles, output_file_path)